In [1]:
# IFC Parser
# Based on https://thinkmoult.com/using-ifcopenshell-parse-ifc-files-python.html

# And: https://github.com/johannesmichael/ifc-python/blob/master/modules/ifc_pset_utils.py


    

In [2]:
import ifcopenshell


In [3]:
ifc_file = ifcopenshell.open('test.ifc')

In [32]:
products = ifc_file.by_type('IfcProduct')
for product in products:
    print(product.is_a())

IfcBuilding
IfcBuildingStorey
IfcBuildingStorey
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWallStandardCase
IfcWindow
IfcWindow
IfcDoor
IfcSlab
IfcSite
IfcOpeningElement
IfcOpeningElement
IfcOpeningElement


In [33]:
for product in products:
    print(product)

#134=IfcBuilding('3DEwd$xVT3Hxc41VY5aNSY',#41,'',$,$,#32,$,'',.ELEMENT.,$,$,$)
#143=IfcBuildingStorey('3DEwd$xVT3Hxc41VXwReda',#41,'Poziom 1',$,'Poziom:8 mm - początek',#141,$,'Poziom 1',.ELEMENT.,0.)
#149=IfcBuildingStorey('3DEwd$xVT3Hxc41VXwRefb',#41,'Poziom 2',$,'Poziom:8 mm - początek',#148,$,'Poziom 2',.ELEMENT.,4000.)
#208=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH2y',#41,'Ściana podstawowa:Ogólne - 200 mm:331325',$,'Ściana podstawowa:Ogólne - 200 mm',#156,#204,'331325')
#390=IfcWallStandardCase('0XKS9_JrzCLBvqx80YyH2y',#41,'Ściana podstawowa:Ogólne - 200 mm:331325',$,'Ściana podstawowa:Ogólne - 200 mm',#347,#388,'331325')
#501=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH3O',#41,'Ściana podstawowa:Ogólne - 200 mm:331353',$,'Ściana podstawowa:Ogólne - 200 mm',#463,#499,'331353')
#593=IfcWallStandardCase('0XKS9_JrzCLBvqx80YyH3O',#41,'Ściana podstawowa:Ogólne - 200 mm:331353',$,'Ściana podstawowa:Ogólne - 200 mm',#553,#591,'331353')
#699=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH07',#41,

In [34]:
wall = ifc_file.by_type('IfcWall')[0]


In [35]:
wall

#208=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH2y',#41,'Ściana podstawowa:Ogólne - 200 mm:331325',$,'Ściana podstawowa:Ogólne - 200 mm',#156,#204,'331325')

In [36]:
print(wall.GlobalId)
print(wall.Name)

0XKS9_JrzCLBvqx8CYyH2y
Ściana podstawowa:Ogólne - 200 mm:331325


In [37]:
wall = ifc_file.by_type('IfcWall')[0]
for definition in wall.IsDefinedBy:
    # To support IFC2X3, we need to filter our results.
    if definition.is_a('IfcRelDefinesByProperties'):
        property_set = definition.RelatingPropertyDefinition
        print(property_set.Name) # Might return Pset_WallCommon

BaseQuantities
Etapy
Inne
Konstrukcje
Wiązania
Wymiary
Pset_QuantityTakeOff
Pset_ReinforcementBarPitchOfWall
Pset_WallCommon


In [38]:
for property in property_set.HasProperties:
    if property.is_a('IfcPropertySingleValue'):
        print(property.Name)
        print(property.NominalValue.wrappedValue)

Reference
Ogólne - 200 mm
IsExternal
True
ExtendToStructure
False
LoadBearing
False


In [39]:
def print_element_quantities(element_quantity):
    for quantity in element_quantity.Quantities:
        print(quantity.Name)
        if quantity.is_a('IfcQuantityArea'):
            print(quantity.AreaValue)

In [40]:
wall = ifc_file.by_type('IfcWall')[0]
for definition in wall.IsDefinedBy:
    related_data = definition.RelatingPropertyDefinition
    if related_data.is_a('IfcPropertySet'):
        pass
    elif related_data.is_a('IfcElementQuantity'):
        print_element_quantities(related_data)

Height
Length
Width
GrossFootprintArea
3.49605381794232
NetVolume
NetSideArea
69998255.8044403
NetSideArea
148.44322028839
NetVolume


AttributeError: entity instance of type 'IfcRelDefinesByType' has no attribute 'RelatingPropertyDefinition'

In [41]:
wall = ifc_file.by_type('IfcWall')[0]
for definition in wall.IsDefinedBy:
    related_data = definition.RelatingPropertyDefinition
    if related_data.is_a('IfcPropertySet'):
        pass
    elif related_data.is_a('IfcElementQuantity'):
        print_element_quantities(related_data)

Height
Length
Width
GrossFootprintArea
3.49605381794232
NetVolume
NetSideArea
69998255.8044403
NetSideArea
148.44322028839
NetVolume


AttributeError: entity instance of type 'IfcRelDefinesByType' has no attribute 'RelatingPropertyDefinition'

In [42]:

def print_quantities(property_definition):
  if 'IfcElementQuantity' == property_definition.is_a():
    for quantity in property_definition.Quantities:
      if 'IfcQuantityArea' == quantity.is_a():
        print('Area value: ' + str(quantity.AreaValue))
      if 'IfcQuantityVolume' == quantity.is_a():
        print('Volume value: ' + str(quantity.VolumeValue))
      if 'IfcQuantityLength' == quantity.is_a():
        print('Length value: ' + str(quantity.LengthValue))

ifc_file = ifcopenshell.open('test.ifc')
products = ifc_file.by_type('IfcSlab')
for product in products:
  if product.IsDefinedBy:
    definitions = product.IsDefinedBy
    for definition in definitions:
      #In IFC2X3 this could be property or type
      #in IFC4 type is in inverse attribute IsTypedBy
      if 'IfcRelDefinesByProperties' == definition.is_a():
        property_definition = definition.RelatingPropertyDefinition
        print_quantities(property_definition)
      if 'IfcRelDefinesByType' == definition.is_a():
        type = definition.RelatingType
        if type.HasPropertySets:
          for property_definition in type.HasPropertySets:
            print_quantities(property_definition)

In [43]:
products

[#1838=IfcSlab('0XKS9_JrzCLBvqv88YyH6j',#41,'Strop:Ogólne 150 mm:331564',$,'Strop:Ogólne 150 mm',#1816,#1836,'331564',.FLOOR.)]

In [7]:
import ifcHelper

In [12]:
ifcHelper

<module 'ifcHelper' from 'C:\\Users\\Mateusz\\Documents\\GitHub\\Ifcparser\\ifcHelper.py'>

In [14]:
walls = ifc_file.by_type('IfcWall')
wall_qnty = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(walls[0])[0])

In [15]:
wall_qnty
wall_qnty['NetSideArea']
wall_area = (0.99 * wall_qnty['NetSideArea']) /2

In [16]:
walls_qnty['NetVolume']
walls_qnty['Width']/1000

NameError: name 'walls_qnty' is not defined

In [ ]:
wall_area = walls_qnty['NetVolume'] / (walls_qnty['Width']/1000)

In [ ]:
wall = list()
for i in range(len(walls)):
    wall.append(get_related_quantities(walls[i]))

In [18]:
walls_netsidearea = list()
for i in range(len(walls)):
    quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(walls[i])[0])
    walls_netsidearea.append(quantities['NetSideArea'])

In [19]:
## Show all the areas
walls_netsidearea #net side area - this is all 3d areas of a wall, we need elevation area

[148.44322028839,
 148.44322028839,
 103.802033989102,
 106.853653989102,
 85.7784791608442,
 85.7784791608442,
 121.376966164673,
 121.554806164673,
 139.769605302175,
 139.769605302175]

In [20]:
status = ifcHelper.get_pset_single_value_by_args(walls[0], 'Pset_WallCommon', 'Status')

NameError: name '_pset_name' is not defined